___
# Ciência dos Dados - PROJETO 2

___
## Alan singer

## Ricardo Peres

## Victor Vazquez

___

___

## 1. Problema

O Classificador Naive-Bayes, o qual se baseia no uso do teorema de Bayes, é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser SPAM considerando as palavras em seu conteúdo e, de forma complementar, permite calcular a probabilidade de uma mensagem ser HAM dada as palavras descritas na mensagem.

Para realizar o MVP (minimum viable product) do projeto, você precisa programar uma versão do classificador que "aprende" o que é uma mensagem SPAM considerando uma base de treinamento e comparar o desempenho dos resultados com uma base de testes. 


___
## 2. Separação da base de dados em Treinamento e Teste

A base de dados deve ser separada em duas partes, aleatoriamente, considerando: 
    
    75% dos dados para a parte Treinamento; e
    25% dos dados para a parte Teste.

In [1]:
import pandas as pd
import os

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/Users/pedrovazquez/Desktop/ciedads/p2


In [3]:
dados = pd.read_excel('spamham2019(1).xlsx')

In [4]:
dados.head(5)

,Email,Class
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


In [5]:
train = dados[int(len(dados)*0.25):]
teste = dados[:int(len(dados)*0.25)]

**Limpando a base de teste:**

In [6]:
teste_email = teste["Email"].str.lower().str.replace(',', '').str.replace('.', '').str.replace('?', '').str.replace('!', '')\
.str.replace('#', '').str.replace('||', '').str.replace('|', '').str.replace('//', '').str.replace('', '')\
.str.replace('"', "").str.replace('\n', "").str.replace('-', "").str.replace(']', "").str.replace('[', "")\
.str.replace('(', "").str.replace(')', "").str.replace('=', "").str.replace('*', "").str.replace('/', "")\
.str.replace('&', "").str.replace('$', "").str.replace('%', "").str.replace('ˆ', "").str.replace('*', "")\
.str.replace(';', "").str.replace(':', "").str.replace('  ', "").str.replace('   ', "").str.replace('    ', "")\
.str.replace('\\', "").str.replace('_', "")
teste = pd.concat([teste_email,teste["Class"]],axis=1)

**Limpando a base de teste e separando em ham e spam:**

In [7]:
train_email = train["Email"].str.lower().str.replace(',', '').str.replace('.', '').str.replace('?', '').str.replace('!', '')\
.str.replace('#', '').str.replace('||', '').str.replace('|', '').str.replace('//', '').str.replace('', '')\
.str.replace('"', "").str.replace('\n', "").str.replace('-', "").str.replace(']', "").str.replace('[', "")\
.str.replace('(', "").str.replace(')', "").str.replace('=', "").str.replace('*', "").str.replace('/', "")\
.str.replace('&', "").str.replace('$', "").str.replace('%', "").str.replace('ˆ', "").str.replace('*', "")\
.str.replace(';', "").str.replace(':', "").str.replace('  ', "").str.replace('   ', "").str.replace('    ', "")\
.str.replace('\\', "").str.replace('_', "")

train_email_limpo = pd.concat([train_email,train["Class"]],axis=1)
train_ham_limpo = train_email_limpo[train_email_limpo.Class == "ham"]
train_spam_limpo = train_email_limpo[train_email_limpo.Class == "spam"]

**Plotando cada base definida e limpa acima**

In [8]:
train_email_limpo.head(5) 

,Email,Class
1393,oh ok,ham
1394,r we still meeting 4 dinner tonight,ham
1395,thats cool i am a gentleman and will treat you...,ham
1396,shall i start from hear,ham
1397,then we wait 4 u lor no need 2 feel bad lar,ham


In [9]:
train_ham_limpo.head(5)

,Email,Class
1393,oh ok,ham
1394,r we still meeting 4 dinner tonight,ham
1395,thats cool i am a gentleman and will treat you...,ham
1396,shall i start from hear,ham
1397,then we wait 4 u lor no need 2 feel bad lar,ham


In [10]:
train_spam_limpo.head(5)

,Email,Class
1406,urgent important information for o2 user today...,spam
1413,dear u've been invited to xchat this is our fi...,spam
1422,congratulations ur awarded either å£500 of cd ...,spam
1429,for salearsenal dartboard good condition but n...,spam
1443,free 1st week entry 2 textpod 4 a chance 2 win...,spam


**Printando o tamanho de cada base:**

In [11]:
print("Base Limpa: {0}".format(len(train_email_limpo)))

Base Limpa: 4179


In [12]:
print("Base Ham: {0}".format(len(train_ham_limpo)))

Base Ham: 3634


In [13]:
print("Base Spam: {0}".format(len(train_spam_limpo)))

Base Spam: 545


___
## 3. Classificador Naive-Bayes

** Com todas as mensagens limpas foram criadas listas de palavras e de frases que foram completas através do for abaixo. Essas listas foram utilizadas para a criação de um dataset das palavras para que a probabilidade de aparição de cada palavra pudesse ser calulada (P(palavra)). E então após o cálculo das probabilidade as mesmas foram adicionadas a base contendo todas as palavras, a quantidade de vezes que cada uma aparece e a sua probabilidade de aparecer. Esses mesmo passos foram realizados para as palavras presentes em Ham e em Spam, para que pudessem ser calculadas as P(palavra/Ham) e P(palavra/Spam), porem somente com as palavras presentes em ham e em spam. **

**Base toda:**

In [35]:
frases = []
palavras = []
for frase in train_email_limpo.Email:
    frases.append(frase.split(" "))
    
for e in frases:
    for palavra in e:
        palavras.append(palavra)
        
len(palavras)

62871

In [33]:
qp_email = pd.DataFrame(data = palavras)
qp_email.columns = ["Palavras"]
q = qp_email["Palavras"].value_counts()
##################################################################################################################
qp_frame = q.to_frame()
qp_frame = q.reset_index()
qp_frame.columns = ["Palavras", "Quantidade"]
qp_frame["Probabilidade"] = 0
###############################################################################################
prob = []
for i in range(len(qp_frame)): 
    prob.append((qp_frame["Quantidade"][i]/len(qp_frame))*100)
###############################################################################################
print("GERAL: ")
qp_frame["Probabilidade"] = prob
qp_frame.head(5)

GERAL: 


,Palavras,Quantidade,Probabilidade
0,to,1649,18.936610
1,i,1575,18.086817
2,you,1508,17.317409
3,a,1059,12.161231
4,the,969,11.127699


**Base ham:**

In [36]:
frases_ham = []
palavras_ham = []
for frase in train_ham_limpo.Email:
    frases_ham.append(frase.split(" "))
for e in frases_ham:
    for palavra in e:
         palavras_ham.append(palavra)

In [37]:
qp_ham_email = pd.DataFrame(data = palavras_ham)
qp_ham_email.columns = ["Palavras"]
q_ham = qp_ham_email["Palavras"].value_counts()
##################################################################################################################
qp_ham_frame = q_ham.to_frame()
qp_ham_frame = q_ham.reset_index()
qp_ham_frame.columns = ["Palavras", "Quantidade"]
qp_ham_frame["Probabilidade"] = 0
###############################################################################################
prob_ham = []
for i in range(len(qp_ham_frame)): 
    prob_ham.append((qp_ham_frame["Quantidade"][i]/len(qp_ham_frame))*100)
###############################################################################################
print("HAM: ")
qp_ham_frame["Probabilidade"] = prob_ham
qp_ham_frame.head(5)

HAM: 


,Palavras,Quantidade,Probabilidade
0,i,1544,22.507289
1,you,1309,19.081633
2,to,1147,16.720117
3,the,818,11.924198
4,a,774,11.282799


**Base spam:**

In [38]:
frases_spam = []
palavras_spam = []
for frase in train_spam_limpo.Email:
    frases_spam.append(frase.split(" "))
for e in frases_spam:
    for palavra in e:
         palavras_spam.append(palavra)

In [39]:
qp_spam_email = pd.DataFrame(data = palavras_spam)
qp_spam_email.columns = ["Palavras"]
q_spam = qp_spam_email["Palavras"].value_counts()
##################################################################################################################
qp_spam_frame = q_spam.to_frame()
qp_spam_frame = q_spam.reset_index()
qp_spam_frame.columns = ["Palavras", "Quantidade"]
qp_spam_frame["Probabilidade"] = 0
###############################################################################################
prob_spam = []
for i in range(len(qp_spam_frame)): 
    prob_spam.append((qp_spam_frame["Quantidade"][i]/len(qp_spam_frame))*100)
###############################################################################################
print("SPAM: ")
qp_spam_frame["Probabilidade"] = prob_spam
qp_spam_frame.head(5)

SPAM: 


,Palavras,Quantidade,Probabilidade
0,to,502,18.476261
1,a,285,10.489510
2,call,256,9.422157
3,your,202,7.434671
4,you,199,7.324255


** Sabendo o tamanho da base e o tamanho da base de spam e de ham é possível calcular a P(ham) e a P(spam), que serão utilizadas posteriormente para o outros cálculos **

In [141]:
prob_spam = (len(train_spam_limpo)*100)/len(train_email_limpo)
prob_ham = (len(train_ham_limpo)*100)/len(train_email_limpo)
print("Tamanhos: Completo = {0}, Ham = {1}, Spam = {2}".format( len(train_email_limpo),len(train_ham_limpo), len(train_spam_limpo)))
print("")
print("Probabilidades: Ham = {0}%, Spam = {1}%".format(prob_ham, prob_spam))

Tamanhos: Completo = 4179, Ham = 3634, Spam = 545

Probabilidades: Ham = 86.95860253649198%, Spam = 13.041397463508016%


**Agora com as probabilidades previamente calculadas é possível descobrir a probabilidade de cada frase e a probabilidade de cada frase dado ham e spam, utilizando o método de laplace em que mesmo se a palavra nao estiver dentro de ham ou de spam a probabilidade nao sera zerada, pois para o calculo da mesma sempre adiciona-se 1 a probabilidade de cada palavra dado ham ou spam.**

p(ham/mensagem)=(p(mensagem/ham)*p(ham))

p(spam/mensagem)=(p(mensgem/spam)*p(spam))


p(mensagem/ham)=(1pm+1)/(len(ham)+len(total_semrepetir)) *(2pm)...)

p(mensagem/spam)=(1pm)+1)/len(spam)+len(total_semrepetir)) *(2pm)...)

In [143]:
def prob_laplanche(frase, typ, palavras):

    frase_nv = frase.split(" ")
    qnt = [1]*len(frase_nv)
    val = 1

    for i in range(len(frase_nv)):
        if frase_nv[i] in typ.Palavras.values.tolist():
            
            qnt[i] += typ.Quantidade[typ.loc[typ["Palavras"] == frase_nv[i]].index[0]]
    
    prob = [i/(len(palavras) + len(qp_frame)) for i in qnt]
    for r in prob: val *= r 
    return val*prob_ham/100 if len(typ) == len(qp_ham_frame) else float(val)*prob_spam/100

In [149]:
def front(teste):
    PF = 0
    FN = 0
    FV = 0
    PV = 0

    for i in range(len(teste)):
        a = prob_laplanche(teste.Email[i], qp_ham_frame, palavras_ham)
        ac = prob_laplanche(teste.Email[i], qp_spam_frame, palavras_spam)

        comp = "ham" if a > ac else "spam"
    
        if comp == "spam" and teste.Class[i] != comp:
            PF += 1 

        if comp == "ham" and teste.Class[i] !=comp:
            FN += 1

        if comp == "ham" and teste.Class[i] == comp:
            FV += 1

        if comp == "spam" and teste.Class[i] == comp:
            PV += 1

        
    return PF,FN,FV,PV, (FV + PV)/len(teste)

In [150]:
front(teste)

(24, 10, 1167, 192, 0.9755922469490309)

In [179]:
len(teste)

1393

## 4. Qualidade do Classificador alterando a base de treinamento

ANALISE DO RESULTADO DA CELULA ACIMA

repetindo todo o processo 10 mil vezes, passo 4 do arquivo do projeto no bb

In [147]:
def back(teste):
    porce = 0
    for i in range(len(teste)):
        
        a = prob_laplanche(teste.Email[i], qp_ham_frame, palavras_ham)
        ac = prob_laplanche(teste.Email[i], qp_spam_frame, palavras_spam)

        comp = "ham" if a > ac else "spam"
    
        if (comp == "spam" and teste.Class[i] == comp) or (comp == "ham" and teste.Class[i] == comp):
            porce += 1

        
    return porce/len(teste)

In [206]:
percentual = []

for i in range(8):
    l = dados.sample(frac=1)
    train_10000 = l[int(len(l)*0.25):].reset_index().drop(['index'], axis=1)
    teste_10000 = l[:int(len(l)*0.25)].reset_index().drop(['index'], axis=1)
    ##########################
    teste_email_10000 = teste_10000["Email"].str.lower().str.replace(',', '').str.replace('.', '').str.replace('?', '').str.replace('!', '')\
    .str.replace('#', '').str.replace('||', '').str.replace('|', '').str.replace('//', '').str.replace('', '')\
    .str.replace('"', "").str.replace('\n', "").str.replace('-', "").str.replace(']', "").str.replace('[', "")\
    .str.replace('(', "").str.replace(')', "").str.replace('=', "").str.replace('*', "").str.replace('/', "")\
    .str.replace('&', "").str.replace('$', "").str.replace('%', "").str.replace('ˆ', "").str.replace('*', "")\
    .str.replace(';', "").str.replace(':', "").str.replace('  ', "").str.replace('   ', "").str.replace('    ', "")\
    .str.replace('\\', "").str.replace('_', "")
    
    teste_10000 = pd.concat([teste_email_10000,teste_10000["Class"]],axis=1)
    ##########################
    train_email_10000 = train_10000["Email"].str.lower().str.replace(',', '').str.replace('.', '').str.replace('?', '').str.replace('!', '')\
    .str.replace('#', '').str.replace('||', '').str.replace('|', '').str.replace('//', '').str.replace('', '')\
    .str.replace('"', "").str.replace('\n', "").str.replace('-', "").str.replace(']', "").str.replace('[', "")\
    .str.replace('(', "").str.replace(')', "").str.replace('=', "").str.replace('*', "").str.replace('/', "")\
    .str.replace('&', "").str.replace('$', "").str.replace('%', "").str.replace('ˆ', "").str.replace('*', "")\
    .str.replace(';', "").str.replace(':', "").str.replace('  ', "").str.replace('   ', "").str.replace('    ', "")\
    .str.replace('\\', "").str.replace('_', "")
    
    train_email_limpo_10000 = pd.concat([train_email_10000,train["Class"]],axis=1)
    train_ham_limpo_10000 = train_email_limpo_10000[train_email_limpo_10000.Class == "ham"]
    train_spam_limpo_10000 = train_email_limpo_10000[train_email_limpo_10000.Class == "spam"]
    ##########################
    frases_spam = []
    palavras_spam = []

    for frase in train_spam_limpo_10000.Email:
        frases_spam.append(str(frase).split(" "))
    for e in frases_spam:
        for palavra in e:
             palavras_spam.append(palavra)

    qp_spam_email = pd.DataFrame(data = palavras_spam)
    qp_spam_email.columns = ["Palavras"]
    q_spam = qp_spam_email["Palavras"].value_counts()
    qp_spam_frame = q_spam.to_frame()
    qp_spam_frame = q_spam.reset_index()
    qp_spam_frame.columns = ["Palavras", "Quantidade"]
    
    ##################
    frases_ham = []
    palavras_ham = []
    for frase in train_ham_limpo_10000.Email:
        frases_ham.append(str(frase).split(" "))
    for e in frases_ham:
        for palavra in e:
             palavras_ham.append(palavra)
            
    qp_ham_email = pd.DataFrame(data = palavras_ham)
    qp_ham_email.columns = ["Palavras"]
    q_ham = qp_ham_email["Palavras"].value_counts()
    qp_ham_frame = q_ham.to_frame()
    qp_ham_frame = q_ham.reset_index()
    qp_ham_frame.columns = ["Palavras", "Quantidade"]
    
    percentual.append(back(teste_10000))

percentual

[0.8463747307968413,
 0.8513998564249821,
 0.8456568557071069,
 0.8377602297200287,
 0.8456568557071069,
 0.8571428571428571,
 0.832735104091888,
 0.8449389806173726]

In [ ]:
plt.hist(percentual)

ANALISE DO RESULTADO E DO HISTOGRAMA, isso que esta dando é o percentual de acerto, porque nessa funcao ele repete tudo por 10 mil vezes pra calcular o percentual de acerto de todas e ver oq mudou*** BREADS LEIA ISSO, E NAO ESQUECE DE LER A DESCRICAO DO TRABALHO QUE ESTA NO BLACK Q E IMPORTANTE PRA VOCE ENTENDER TUDO PARA ANALISAR 